# Classification of the Feature Neurons from the SNN
This notebook performs the classification of the feature neurons from the SNN according to certain criteria. In the future, it would be interesting to implement a 3rd layer in the SNN to classify the feature neurons automatically. For now, this manual classification will classify each neuron as one of the following:
- **Silent Neuron**: Neuron that does not fire at all.
- **Noisy Neuron**: Neuron that fires randomly or without a relevant pattern.
- **Ripple Neuron**: Neuron that fires in the presence of a ripple.
- **Fast Ripple Neuron**: Neuron that fires in the presence of a fast ripple.

## Check WD (change if necessary) and file loading

In [17]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/snn" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/snn"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/snn


## Load the Voltage and Current Dynamics during the SNN run

In [18]:
import numpy as np
from utils.io import preview_np_array
from utils.input import MarkerType, band_to_file_name

# Declare if using ripples, fast ripples, or both
chosen_band = MarkerType.FAST_RIPPLE     # RIPPLE, FAST_RIPPLE, or BOTH
band_file_name = band_to_file_name(chosen_band)

INPUT_PATH = f"./results/custom_subset_90-119_segment500_200"

# Load the voltage and current data from the numpy files
voltage_file_name = f"{INPUT_PATH}/{band_file_name}_v_dynamics_0.07dv_5ch_time1000-300-1.npy"
current_file_name = f"{INPUT_PATH}/{band_file_name}_u_dynamics_0.07dv_5ch_time1000-300-1.npy"

v_dynamics = np.load(voltage_file_name)
u_dynamics = np.load(current_file_name)

preview_np_array(v_dynamics, "v_dynamics", edge_items=3)
preview_np_array(u_dynamics, "u_dynamics", edge_items=3)

v_dynamics Shape: (300, 256).
Preview: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-3.00000000e-01 -3.00000000e-01 -3.00000000e-01 ... -3.00000000e-01
  -3.00000000e-01 -3.00000000e-01]
 [-3.63637806e-02 -6.19632354e-02  2.10000000e-02 ... -1.82933897e-01
   1.99834875e-02 -1.20341243e-01]
 ...
 [ 1.08405053e-09  9.42952115e-10 -1.23785429e-10 ...  1.94098992e-09
   7.40639226e-10  2.13210245e-10]
 [ 1.00816699e-09  8.76945467e-10 -1.15120449e-10 ...  1.80512063e-09
   6.88794480e-10  1.98285528e-10]
 [ 9.37595301e-10  8.15559284e-10 -1.07062018e-10 ...  1.67876218e-09
   6.40578866e-10  1.84405541e-10]]
u_dynamics Shape: (300, 256).
Preview: [[ 0.00000000e+000  0.00000000e+000  0.00000000e+000 ...  0.00000000e+000
   0.00000000e+000  0.00000000e+000]
 [-3.00000000e-001 -3.00000000e-001 -3.00000000e-001 ... -3.00000000e-001
  -3.00000000e-001 -3.00000000e-001]
 [ 2.42636219e-001  2.17036765e-001  3.00000000e-001 ...  9.6066

## Load the SNN Configuration and extract the fields
The SNN configuration contains:
- Numpy array with the ground_truth for each timestep
- Initial Time Offset
- Virtual Time Step Interval
- Number of Steps

In [19]:
from utils.snn import SNNSimConfig

# Load the SNN Config data
snn_config_file_name = f"{INPUT_PATH}/{band_file_name}_snn_config.npy"

# Load the SNNConfig data as an element of the class SNNConfig
snn_config: SNNSimConfig = np.load(snn_config_file_name, allow_pickle=True).item()

# Extract the data fields from the SNNConfig object
ground_truth = snn_config.ground_truth
init_offset = snn_config.init_offset
virtual_time_step_interval = snn_config.virtual_time_step_interval
num_steps = snn_config.num_steps

preview_np_array(ground_truth, "ground_truth", edge_items=3)
np.count_nonzero(ground_truth)

print("init_offset: ", init_offset)
print("virtual_time_step_interval: ", virtual_time_step_interval)
print("num_steps: ", num_steps)

ground_truth Shape: (245760,).
Preview: [0. 0. 0. ... 0. 0. 0.]
init_offset:  1000
virtual_time_step_interval:  1
num_steps:  300


## Find the timesteps where the network spiked
Let's find the timesteps where the network spiked and create a `dictionary` mapping each feature neuron to the timesteps where it spiked.

In [22]:
from utils.data_analysis import find_spike_times

# Create a map storing the spike times for each feature neuron
neuron_spike_times = {}

# Call the find_spike_times util function that detects the spikes in a voltage array
spike_times_lif1 = find_spike_times(v_dynamics, u_dynamics)

for (spike_time, neuron_idx) in spike_times_lif1:
    # Calculate the spike time in ms
    real_spike_time = init_offset + spike_time * virtual_time_step_interval

    # If the neuron index is not in the map (first time the feature neuron spikes), add it
    if neuron_idx not in neuron_spike_times:
        neuron_spike_times[neuron_idx] = [real_spike_time]
    # Otherwise, append the spike time to the list of spike times for that neuron
    else:
        neuron_spike_times[neuron_idx].append(real_spike_time)

    # print(f"Spike time: {real_spike_time} (iter. {spike_time}) at neuron: {neuron_idx}")

# Print the spike times for each feature neuron
print("neuron_spike_times: ", neuron_spike_times)


neuron_spike_times:  {0: [1007, 1017], 31: [1008], 5: [1009], 4: [1010], 47: [1011], 50: [1014], 121: [1018]}
